# DMTA Relaxation Domain: E(t) Stress Relaxation

**Learning Objectives:**
- Understand stress relaxation E(t) measurement in DMTA
- Convert between E(t) and G(t) using Poisson ratio
- Fit Prony series models to relaxation data
- Extract characteristic relaxation times from fitted parameters

**Estimated Time:** 5-7 minutes (FAST_MODE), 10-12 minutes (full)

**Key Concepts:**
- Relaxation modulus E(t) = E_inf + Σ E_k exp(-t/τ_k)
- Relation to shear: E(t) = 2(1+ν)G(t)
- Same Prony decomposition as oscillation domain
- Material properties independent of test geometry

In [1]:
import gc
import os
import sys
import warnings

import matplotlib.pyplot as plt
import numpy as np

if os.path.abspath(os.path.join(os.getcwd(), '../..')) not in sys.path:
    sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '../..')))

from rheojax.core.jax_config import safe_import_jax

jax, jnp = safe_import_jax()

np.random.seed(42)
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 11
warnings.filterwarnings('ignore', category=RuntimeWarning)

FAST_MODE = os.environ.get('FAST_MODE', '1') == '1'
print(f'FAST_MODE: {FAST_MODE}')

FAST_MODE: True


## 1. Physics of Stress Relaxation

In a stress relaxation test, a constant strain γ₀ is applied instantaneously, and the stress σ(t) is measured as it decays over time. The relaxation modulus is:

$$E(t) = \frac{\sigma(t)}{\gamma_0}$$

For viscoelastic materials, E(t) follows a **Prony series** decomposition:

$$E(t) = E_{\infty} + \sum_{k=1}^{N} E_k \exp\left(-\frac{t}{\tau_k}\right)$$

where:
- E∞ = equilibrium modulus (long-time plateau)
- E_k = relaxation strength of mode k
- τ_k = relaxation time of mode k

**Relation to Shear Modulus:**

For isotropic materials, the tensile relaxation modulus E(t) and shear relaxation modulus G(t) are related by:

$$E(t) = 2(1 + \nu) G(t)$$

where ν is Poisson's ratio. For incompressible rubber-like materials (ν ≈ 0.5), E(t) ≈ 3 G(t).

**DMTA Capabilities:**
- Modern DMTA instruments can perform step-strain tests in tension mode
- Typical strain γ₀ = 0.001 - 0.01 (linear regime)
- Time range: 0.001 s to 1000 s
- Temperature control for time-temperature superposition

## 2. Generate Synthetic Relaxation Data

We'll create a 3-mode Prony series representing a viscoelastic polymer:
- Fast mode: τ₁ = 0.1 s (β relaxation, local chain motion)
- Medium mode: τ₂ = 1.0 s (segmental relaxation)
- Slow mode: τ₃ = 10 s (reptation/terminal relaxation)
- Rubber plateau: E∞ = 50 kPa (crosslinked network)

In [2]:
# Define 3-mode Prony series in G-space (shear modulus)
G_inf = 5e4  # Pa, equilibrium shear modulus
G_modes = np.array([1e6, 5e5, 2e5])  # Pa
tau_modes = np.array([0.1, 1.0, 10.0])  # s

# Poisson ratio for incompressible rubber
nu = 0.5
conversion_factor = 2 * (1 + nu)  # E = 2(1+ν)G

# Time vector
n_points = 80 if FAST_MODE else 150
t = np.logspace(-3, 3, n_points)  # 0.001 s to 1000 s

# Generate G(t) relaxation modulus
G_t_exact = G_inf + np.sum(
    G_modes[:, None] * np.exp(-t[None, :] / tau_modes[:, None]), axis=0
)

# Convert to E(t) = 2(1+ν)G(t)
E_t_exact = conversion_factor * G_t_exact

# Add 2% Gaussian noise
noise_level = 0.02
noise = np.random.normal(0, noise_level * np.mean(E_t_exact), size=len(t))
E_t = E_t_exact + noise

# Print synthetic material properties
print("Synthetic Material Properties (G-space):")
print(f"  G∞ = {G_inf:.2e} Pa")
for i, (G_k, tau_k) in enumerate(zip(G_modes, tau_modes), 1):
    print(f"  Mode {i}: G_{i} = {G_k:.2e} Pa, τ_{i} = {tau_k:.2f} s")
print(f"\nConversion to E-space (ν = {nu}):")
print(f"  E∞ = {conversion_factor * G_inf:.2e} Pa")
print(f"  E_k = {conversion_factor} × G_k")
print(f"\nData points: {len(t)}")
print(f"Noise level: {noise_level*100}%")

Synthetic Material Properties (G-space):
  G∞ = 5.00e+04 Pa
  Mode 1: G_1 = 1.00e+06 Pa, τ_1 = 0.10 s
  Mode 2: G_2 = 5.00e+05 Pa, τ_2 = 1.00 s
  Mode 3: G_3 = 2.00e+05 Pa, τ_3 = 10.00 s

Conversion to E-space (ν = 0.5):
  E∞ = 1.50e+05 Pa
  E_k = 3.0 × G_k

Data points: 80
Noise level: 2.0%


## 3. Visualize Synthetic Data

The relaxation curve shows:
1. **Short-time plateau** (glassy modulus) ~ 1.5 MPa
2. **Fast decay** (β relaxation)
3. **Intermediate plateau** (entanglement network)
4. **Slow decay** (terminal relaxation)
5. **Long-time plateau** (E∞, rubber modulus) ~ 150 kPa

In [3]:
fig, ax = plt.subplots()
ax.loglog(t, E_t, 'o', label='Synthetic E(t) data', markersize=4, alpha=0.7)
ax.loglog(t, E_t_exact, 'k--', label='True E(t)', linewidth=1.5)
ax.set_xlabel('Time (s)')
ax.set_ylabel('E(t) (Pa)')
ax.set_title('Synthetic Stress Relaxation Data (3-Mode Prony Series)')
ax.legend()
ax.grid(True, alpha=0.3)
plt.close('all')

# Compute relaxation time span
tau_min, tau_max = tau_modes.min(), tau_modes.max()
print(f"Relaxation time span: {tau_min} s to {tau_max} s ({tau_max/tau_min:.0f}× decade range)")

Relaxation time span: 0.1 s to 10.0 s (100× decade range)


## 4. Fit Single-Mode Maxwell Model

The Maxwell model is the simplest viscoelastic relaxation model:

$$E(t) = E_0 \exp\left(-\frac{t}{\tau}\right)$$

where τ = η/E₀ is the relaxation time. This model:
- Captures single-mode relaxation
- Cannot represent equilibrium plateau (E∞)
- Will fit to dominant relaxation mode in multi-mode data

In [4]:
from rheojax.models.classical.maxwell import Maxwell

# Initialize Maxwell model
maxwell = Maxwell()

# Fit to E(t) data with tension mode
maxwell.fit(
    t,
    E_t,
    test_mode='relaxation',
    deformation_mode='tension',
    poisson_ratio=nu,
)

# Predict E(t)
E_maxwell = maxwell.predict(t, test_mode='relaxation')

# Compute R²
ss_res = np.sum((E_t - E_maxwell) ** 2)
ss_tot = np.sum((E_t - np.mean(E_t)) ** 2)
r2_maxwell = 1 - ss_res / ss_tot

print("Maxwell Model Fit Results:")
print(f"  R² = {r2_maxwell:.4f}")
print(f"\nFitted Parameters (G-space):")
G0_fit = maxwell.parameters['G0'].value
eta_fit = maxwell.parameters['eta'].value
tau_maxwell = eta_fit / G0_fit
print(f"  G₀ = {G0_fit:.2e} Pa")
print(f"  η = {eta_fit:.2e} Pa·s")
print(f"  τ = η/G₀ = {tau_maxwell:.2f} s")
print(f"\nCorresponding E-space:")
print(f"  E₀ = {conversion_factor * G0_fit:.2e} Pa")
print(f"  (Same τ = {tau_maxwell:.2f} s)")

INFO:nlsq.least_squares:Starting least squares optimization method=trf | n_params=2 | loss=linear | ftol=1.0000e-06 | xtol=1.0000e-06 | gtol=1.0000e-06


PERFORMANCE:nlsq.least_squares:Timer: optimization elapsed=0.826344s


INFO:nlsq.least_squares:Convergence reason=`ftol` termination condition is satisfied. | iterations=27 | final_cost=17.4596 | elapsed=0.826s | final_gradient_norm=6882.2748


Maxwell Model Fit Results:
  R² = 0.7330

Fitted Parameters (G-space):
  G₀ = 9.76e+05 Pa
  η = 1.52e+06 Pa·s
  τ = η/G₀ = 1.56 s

Corresponding E-space:
  E₀ = 2.93e+06 Pa
  (Same τ = 1.56 s)


## 5. Fit Zener Model (2-Mode)

The Zener (Standard Linear Solid) model adds an equilibrium modulus:

$$E(t) = E_{\infty} + E_1 \exp\left(-\frac{t}{\tau_1}\right)$$

This model:
- Captures single relaxation mode + equilibrium plateau
- Better suited for crosslinked networks (rubber)
- Cannot resolve multiple relaxation times

In [5]:
from rheojax.models.classical.zener import Zener

# Initialize Zener model
zener = Zener()

# Fit to E(t) data
zener.fit(
    t,
    E_t,
    test_mode='relaxation',
    deformation_mode='tension',
    poisson_ratio=nu,
)

# Predict E(t)
E_zener = zener.predict(t, test_mode='relaxation')

# Compute R²
ss_res = np.sum((E_t - E_zener) ** 2)
ss_tot = np.sum((E_t - np.mean(E_t)) ** 2)
r2_zener = 1 - ss_res / ss_tot

print("Zener Model Fit Results:")
print(f"  R² = {r2_zener:.4f}")
print(f"\nFitted Parameters (G-space):")
Ge_fit = zener.parameters['Ge'].value
Gm_fit = zener.parameters['Gm'].value
eta_fit_z = zener.parameters['eta'].value
tau_zener = eta_fit_z / Gm_fit
print(f"  Ge (equilibrium) = {Ge_fit:.2e} Pa")
print(f"  Gm (Maxwell arm) = {Gm_fit:.2e} Pa")
print(f"  eta = {eta_fit_z:.2e} Pa·s")
print(f"  tau = eta/Gm = {tau_zener:.2f} s")
print(f"\nCorresponding E-space:")
print(f"  E_inf = {conversion_factor * Ge_fit:.2e} Pa")
print(f"  E1 = {conversion_factor * Gm_fit:.2e} Pa")
print(f"\nComparison to True Values:")
print(f"  True G_inf = {G_inf:.2e} Pa, Error = {100*(Ge_fit - G_inf)/G_inf:.1f}%")
print(f"  Dominant tau_true ~ {tau_modes[1]:.2f} s, tau_fit = {tau_zener:.2f} s")

INFO:nlsq.least_squares:Starting least squares optimization method=trf | n_params=3 | loss=linear | ftol=1.0000e-06 | xtol=1.0000e-06 | gtol=1.0000e-06


PERFORMANCE:nlsq.least_squares:Timer: optimization elapsed=0.635829s


INFO:nlsq.least_squares:Convergence reason=`ftol` termination condition is satisfied. | iterations=25 | final_cost=7.1618 | elapsed=0.636s | final_gradient_norm=4.1525


Zener Model Fit Results:
  R² = 0.8294

Fitted Parameters (G-space):
  Ge (equilibrium) = 3.67e+04 Pa
  Gm (Maxwell arm) = 1.13e+06 Pa
  eta = 1.33e+06 Pa·s
  tau = eta/Gm = 1.17 s

Corresponding E-space:
  E_inf = 1.10e+05 Pa
  E1 = 3.40e+06 Pa

Comparison to True Values:
  True G_inf = 5.00e+04 Pa, Error = -26.6%
  Dominant tau_true ~ 1.00 s, tau_fit = 1.17 s


## 6. Compare Models

**Maxwell vs Zener:**
- Maxwell: Fits to dominant mode, cannot capture E∞
- Zener: Captures equilibrium plateau + dominant relaxation time
- For multi-mode materials, Generalized Maxwell (N-mode Prony series) is needed

In [6]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Left: E(t) comparison
ax1.loglog(t, E_t, 'o', label='Data', markersize=4, alpha=0.6)
ax1.loglog(t, E_t_exact, 'k--', label='True (3-mode)', linewidth=2)
ax1.loglog(t, E_maxwell, '-', label=f'Maxwell (R²={r2_maxwell:.3f})', linewidth=1.5)
ax1.loglog(t, E_zener, '-', label=f'Zener (R²={r2_zener:.3f})', linewidth=1.5)
ax1.set_xlabel('Time (s)')
ax1.set_ylabel('E(t) (Pa)')
ax1.set_title('Relaxation Modulus Fit Comparison')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Right: Residuals
residual_maxwell = E_t - E_maxwell
residual_zener = E_t - E_zener
ax2.semilogx(t, residual_maxwell / E_t * 100, 'o-', label='Maxwell', markersize=3)
ax2.semilogx(t, residual_zener / E_t * 100, 's-', label='Zener', markersize=3)
ax2.axhline(0, color='k', linestyle='--', linewidth=1)
ax2.set_xlabel('Time (s)')
ax2.set_ylabel('Relative Error (%)')
ax2.set_title('Fit Residuals')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.close('all')

print("\nKey Observations:")
print(f"  1. Maxwell fails to capture long-time plateau (E∞)")
print(f"  2. Zener captures equilibrium modulus but averages multiple modes")
print(f"  3. Residuals show systematic deviations due to unresolved modes")
print(f"  4. For accurate multi-mode fitting, use Generalized Maxwell")


Key Observations:
  1. Maxwell fails to capture long-time plateau (E∞)
  2. Zener captures equilibrium modulus but averages multiple modes
  3. Residuals show systematic deviations due to unresolved modes
  4. For accurate multi-mode fitting, use Generalized Maxwell


## 7. Knowledge Extraction: Relaxation Spectrum

From fitted Zener model, we extract:
- **Characteristic relaxation time**: τ = η_m/G_m
- **Relaxation strength ratio**: E₁/E∞ (mobility of relaxing segments)
- **Material state**: E∞ > 0 → rubber/gel (permanent network)

In [7]:
# Extract knowledge from Zener fit
E_inf_fit = conversion_factor * Ge_fit
E1_fit = conversion_factor * Gm_fit
relaxation_strength_ratio = E1_fit / E_inf_fit

print("Knowledge Extraction from Zener Model:")
print("\n1. Relaxation Time:")
print(f"   tau = {tau_zener:.2f} s")
print(f"   -> Dominant molecular rearrangement timescale")
print(f"   -> True multi-mode range: {tau_min} - {tau_max} s")
print("\n2. Relaxation Strength:")
print(f"   E1/E_inf = {relaxation_strength_ratio:.2f}")
print(f"   -> {relaxation_strength_ratio:.1f}x modulus drop during relaxation")
print(f"   -> High ratio indicates significant chain mobility")
print("\n3. Material Classification:")
print(f"   E_inf = {E_inf_fit:.2e} Pa > 0 -> RUBBER (permanent network)")
print(f"   -> Crosslinked elastomer with relaxing entanglements")
print("\n4. Physical Interpretation:")
print(f"   - Fast modes (tau < {tau_zener:.1f} s): local chain motion")
print(f"   - Dominant mode (tau ~ {tau_zener:.1f} s): segmental relaxation")
print(f"   - Slow modes (tau > {tau_zener:.1f} s): entanglement relaxation")
print(f"   - Plateau (E_inf): permanent chemical crosslinks")

print("\n5. Recommended Analysis:")
print(f"   - Use Generalized Maxwell for full spectrum H(tau)")
print(f"   - Perform TTS to extend time range")
print(f"   - Compare to SAOS: G' plateau should match {conversion_factor * Ge_fit:.2e} Pa")

Knowledge Extraction from Zener Model:

1. Relaxation Time:
   tau = 1.17 s
   -> Dominant molecular rearrangement timescale
   -> True multi-mode range: 0.1 - 10.0 s

2. Relaxation Strength:
   E1/E_inf = 30.91
   -> 30.9x modulus drop during relaxation
   -> High ratio indicates significant chain mobility

3. Material Classification:
   E_inf = 1.10e+05 Pa > 0 -> RUBBER (permanent network)
   -> Crosslinked elastomer with relaxing entanglements

4. Physical Interpretation:
   - Fast modes (tau < 1.2 s): local chain motion
   - Dominant mode (tau ~ 1.2 s): segmental relaxation
   - Slow modes (tau > 1.2 s): entanglement relaxation
   - Plateau (E_inf): permanent chemical crosslinks

5. Recommended Analysis:
   - Use Generalized Maxwell for full spectrum H(tau)
   - Perform TTS to extend time range
   - Compare to SAOS: G' plateau should match 1.10e+05 Pa


## 8. Connection to Oscillation Domain

**Fundamental Relation:**

The same Prony series that describes E(t) also determines the storage and loss moduli:

$$E'(\omega) = E_{\infty} + \sum_{k=1}^{N} E_k \frac{(\omega \tau_k)^2}{1 + (\omega \tau_k)^2}$$

$$E''(\omega) = \sum_{k=1}^{N} E_k \frac{\omega \tau_k}{1 + (\omega \tau_k)^2}$$

**Key Insights:**
- Relaxation times τ_k are material properties (independent of test geometry)
- E(t) → E'(ω), E''(ω) via Fourier transform
- DMTA can measure both domains on same sample
- Consistency check: E∞ from relaxation should match E'(ω→0) plateau

In [8]:
# Predict SAOS from fitted Zener model
omega = np.logspace(-3, 3, 100)
E_star = zener.predict(omega, test_mode='oscillation')

fig, ax = plt.subplots()
ax.loglog(omega, np.abs(E_star), '-', label="E* magnitude", linewidth=2)
ax.loglog(omega, E_star.real, '--', label="E' (storage)", linewidth=1.5)
ax.loglog(omega, E_star.imag, '--', label='E" (loss)', linewidth=1.5)
ax.axhline(E_inf_fit, color='k', linestyle=':', label=f'E∞ = {E_inf_fit:.2e} Pa')
ax.set_xlabel('Angular Frequency (rad/s)')
ax.set_ylabel('Modulus (Pa)')
ax.set_title('Predicted SAOS from Zener Relaxation Fit')
ax.legend()
ax.grid(True, alpha=0.3)
plt.close('all')

print("Consistency Checks:")
E_prime_plateau = E_star.real[-1]  # Low-frequency limit
print(f"  E'(ω→0) = {E_prime_plateau:.2e} Pa")
print(f"  E∞ (relaxation) = {E_inf_fit:.2e} Pa")
print(f"  Agreement: {100*(1 - abs(E_prime_plateau - E_inf_fit)/E_inf_fit):.1f}%")
print(f"\nNote: E'(ω→0) = E∞ is a fundamental requirement of linear viscoelasticity")

Consistency Checks:
  E'(ω→0) = 3.51e+06 Pa
  E∞ (relaxation) = 1.10e+05 Pa
  Agreement: -2991.1%

Note: E'(ω→0) = E∞ is a fundamental requirement of linear viscoelasticity


## 9. Fitting Real Relaxation Master Curve

The `data/` directory contains a real relaxation master curve from the [pyvisco](https://github.com/NREL/pyvisco) project spanning ~31 decades in time (TTS-expanded). We fit a Generalized Maxwell model and compare to the reference Prony series.

In [9]:
import pandas as pd

from rheojax.models.multimode.generalized_maxwell import GeneralizedMaxwell

# Load real relaxation master curve
data_dir = os.path.join(os.path.dirname(os.path.abspath('.')), 'dmta', 'data')
if not os.path.exists(data_dir):
    data_dir = os.path.join('.', 'data')

df_relax = pd.read_csv(os.path.join(data_dir, 'time_user_master.csv'), skiprows=[1])
df_relax.columns = df_relax.columns.str.strip()
t_real = df_relax['t'].values
E_real_MPa = df_relax['E_relax'].values
E_real_Pa = E_real_MPa * 1e6  # MPa -> Pa

print(f'Real relaxation data: {len(t_real)} points')
print(f'Time range: {t_real.min():.2e} - {t_real.max():.2e} s ({np.log10(t_real.max()/t_real.min()):.1f} decades)')
print(f'E_glassy ~ {E_real_MPa.max():.0f} MPa, E_rubbery ~ {E_real_MPa.min():.0f} MPa')

# Visualize
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

ax1.loglog(t_real, E_real_MPa, 'b-', lw=2, alpha=0.8)
ax1.set_xlabel('Time (s)')
ax1.set_ylabel('E(t) (MPa)')
ax1.set_title('Real Relaxation Master Curve (31 decades)')
ax1.grid(True, alpha=0.3)

# Identify regions
ax2.semilogx(t_real, E_real_MPa, 'b-', lw=2, alpha=0.8)
ax2.axhline(E_real_MPa.max(), color='r', ls=':', alpha=0.5, label=f'E_glassy ~ {E_real_MPa.max():.0f} MPa')
ax2.axhline(E_real_MPa.min(), color='g', ls=':', alpha=0.5, label=f'E_rubbery ~ {E_real_MPa.min():.0f} MPa')
ax2.set_xlabel('Time (s)')
ax2.set_ylabel('E(t) (MPa)')
ax2.set_title('Identifying Glassy and Rubbery Plateaus')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.close('all')

Real relaxation data: 481 points
Time range: 2.82e-03 - 1.39e+28 s (30.7 decades)
E_glassy ~ 1714 MPa, E_rubbery ~ 86 MPa


In [10]:
# Fit Generalized Maxwell to real relaxation data
n_modes_relax = 5 if FAST_MODE else 10

gmm_relax = GeneralizedMaxwell(n_modes=n_modes_relax, modulus_type='tensile')

gmm_relax.fit(
    t_real, E_real_Pa,
    test_mode='relaxation',
    optimization_factor=None,
)

E_gmm_relax = gmm_relax.predict(t_real, test_mode='relaxation')

# R-squared
ss_res_r = np.sum((E_real_Pa - E_gmm_relax)**2)
ss_tot_r = np.sum((E_real_Pa - np.mean(E_real_Pa))**2)
R2_relax = 1 - ss_res_r / ss_tot_r

print(f'GMM fit ({gmm_relax._n_modes} active modes):')
print(f'  R2 = {R2_relax:.6f}')

# Plot fit
fig, ax = plt.subplots(figsize=(10, 6))
ax.loglog(t_real, E_real_MPa, 'bo', ms=3, alpha=0.4, label='Real E(t)')
ax.loglog(t_real, E_gmm_relax / 1e6, 'r-', lw=2, label=f'GMM fit ({gmm_relax._n_modes} modes)')
ax.set_xlabel('Time (s)')
ax.set_ylabel('E(t) (MPa)')
ax.set_title(f'Generalized Maxwell Fit to Real Relaxation (R2={R2_relax:.4f})')
ax.legend()
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.close('all')

INFO:nlsq.least_squares:Starting least squares optimization method=trf | n_params=11 | loss=linear | ftol=1.0000e-06 | xtol=1.0000e-06 | gtol=1.0000e-06


PERFORMANCE:nlsq.least_squares:Timer: optimization elapsed=1.128851s


INFO:nlsq.least_squares:Convergence reason=`ftol` termination condition is satisfied. | iterations=11 | final_cost=4.7268e+19 | elapsed=1.129s | final_gradient_norm=2.6377e+20


GMM fit (5 active modes):
  R2 = 0.435170


In [11]:
# Compare fitted Prony terms to pyvisco reference
df_prony = pd.read_csv(os.path.join(data_dir, 'prony_terms_reference.csv'), skiprows=[1])
df_prony.columns = df_prony.columns.str.strip()
tau_ref = df_prony['tau_i'].values
E_ref_MPa = df_prony['E_i'].values

# Extract fitted Prony terms
prefix = 'E' if gmm_relax._modulus_type == 'tensile' else 'G'
tau_fit_r = []
E_fit_r = []
for k in range(gmm_relax._n_modes):
    E_k = gmm_relax.parameters.get_value(f'{prefix}_{k+1}')
    tau_k = gmm_relax.parameters.get_value(f'tau_{k+1}')
    if E_k > 1e-6:
        tau_fit_r.append(tau_k)
        E_fit_r.append(E_k / 1e6)  # Pa -> MPa

tau_fit_r = np.array(tau_fit_r) if tau_fit_r else np.array([])
E_fit_r = np.array(E_fit_r) if E_fit_r else np.array([])
E_inf_r = gmm_relax.parameters.get_value('E_inf' if prefix == 'E' else 'G_inf') / 1e6

print(f'Reference: {len(tau_ref)} modes, E0 = {E_ref_MPa.sum():.0f} MPa')
print(f'Fitted: {len(tau_fit_r)} modes, E_inf = {E_inf_r:.1f} MPa')

# Relaxation spectrum bar chart
fig, ax = plt.subplots(figsize=(10, 5))
ax.bar(np.log10(tau_ref), E_ref_MPa, width=0.3, alpha=0.5, label=f'Reference ({len(tau_ref)} modes)', color='steelblue')
if len(tau_fit_r) > 0:
    ax.bar(np.log10(tau_fit_r), E_fit_r, width=0.2, alpha=0.7, label=f'RheoJAX GMM ({len(tau_fit_r)} modes)', color='coral')
ax.set_xlabel('log10(tau / s)')
ax.set_ylabel('E_k (MPa)')
ax.set_title('Discrete Relaxation Spectrum: Reference vs Fitted')
ax.legend()
plt.tight_layout()
plt.close('all')

Reference: 30 modes, E0 = 25685 MPa
Fitted: 5 modes, E_inf = 686.4 MPa


In [12]:
# Fit single-mode Zener to show limitation on broad spectrum
zener_real = Zener()
zener_real.fit(t_real, E_real_Pa, test_mode='relaxation', deformation_mode='tension', poisson_ratio=0.5)
E_zener_real = zener_real.predict(t_real, test_mode='relaxation')

ss_res_z = np.sum((E_real_Pa - E_zener_real)**2)
ss_tot_z = np.sum((E_real_Pa - np.mean(E_real_Pa))**2)
R2_zener_real = 1 - ss_res_z / ss_tot_z

fig, ax = plt.subplots(figsize=(10, 6))
ax.loglog(t_real, E_real_MPa, 'bo', ms=3, alpha=0.3, label='Real data')
ax.loglog(t_real, E_gmm_relax / 1e6, 'r-', lw=2, label=f'GMM (R2={R2_relax:.4f})')
ax.loglog(t_real, E_zener_real / 1e6, 'g--', lw=2, label=f'Zener (R2={R2_zener_real:.4f})')
ax.set_xlabel('Time (s)')
ax.set_ylabel('E(t) (MPa)')
ax.set_title('GMM vs Zener on Real 31-Decade Relaxation')
ax.legend()
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.close('all')

print(f'GMM R2 = {R2_relax:.4f} ({gmm_relax._n_modes} modes)')
print(f'Zener R2 = {R2_zener_real:.4f} (1 mode)')
print(f'Single-mode Zener cannot capture the broad relaxation of a real polymer.')

INFO:nlsq.least_squares:Starting least squares optimization method=trf | n_params=3 | loss=linear | ftol=1.0000e-06 | xtol=1.0000e-06 | gtol=1.0000e-06


PERFORMANCE:nlsq.least_squares:Timer: optimization elapsed=0.305954s


INFO:nlsq.least_squares:Convergence reason=`ftol` termination condition is satisfied. | iterations=22 | final_cost=95.4258 | elapsed=0.306s | final_gradient_norm=1.3670e-05


GMM R2 = 0.4352 (5 modes)
Zener R2 = -0.6470 (1 mode)
Single-mode Zener cannot capture the broad relaxation of a real polymer.


In [13]:
# Cross-domain validation: predict E*(omega) from relaxation-fitted Prony
if not FAST_MODE:
    df_freq = pd.read_csv(os.path.join(data_dir, 'freq_user_master.csv'), skiprows=[1])
    omega_freq = 2 * np.pi * df_freq['f'].values
    E_stor_freq = df_freq['E_stor'].values * 1e6  # MPa -> Pa
    E_loss_freq = df_freq['E_loss'].values * 1e6

    # Predict oscillation from relaxation-fitted model
    E_pred_osc = gmm_relax.predict(omega_freq, test_mode='oscillation')
    E_pred_prime = E_pred_osc[:, 0]
    E_pred_double = E_pred_osc[:, 1]

    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

    ax1.loglog(omega_freq, E_stor_freq / 1e6, 'ro', ms=3, alpha=0.5, label="E' measured")
    ax1.loglog(omega_freq, E_pred_prime / 1e6, 'r-', lw=2, label="E' from relaxation Prony")
    ax1.set_xlabel(chr(969) + ' (rad/s)')
    ax1.set_ylabel("E' (MPa)")
    ax1.set_title("Cross-domain: E'(omega) from Relaxation Fit")
    ax1.legend()
    ax1.grid(True, alpha=0.3)

    ax2.loglog(omega_freq, E_loss_freq / 1e6, 'bs', ms=3, alpha=0.5, label='E" measured')
    ax2.loglog(omega_freq, E_pred_double / 1e6, 'b-', lw=2, label='E" from relaxation Prony')
    ax2.set_xlabel(chr(969) + ' (rad/s)')
    ax2.set_ylabel('E" (MPa)')
    ax2.set_title('Cross-domain: E"(omega) from Relaxation Fit')
    ax2.legend()
    ax2.grid(True, alpha=0.3)

    plt.tight_layout()
    plt.close('all')

    # R2 for cross-domain
    ss_res_cross = np.sum((E_stor_freq - E_pred_prime)**2)
    ss_tot_cross = np.sum((E_stor_freq - np.mean(E_stor_freq))**2)
    R2_cross = 1 - ss_res_cross / ss_tot_cross
    print(f"Cross-domain R2(E') = {R2_cross:.4f}")
    print(f'Same Prony series describes BOTH E(t) and E*(omega) - a fundamental result.')
else:
    print('Cross-domain validation skipped in FAST_MODE (run with FAST_MODE=0 for full analysis)')

Cross-domain validation skipped in FAST_MODE (run with FAST_MODE=0 for full analysis)


## Key Takeaways

1. **E(t) = 2(1+nu)G(t)** — RheoJAX handles conversion automatically via `deformation_mode='tension'`
2. **Real polymer relaxation** spans 31 decades (TTS-expanded) with glassy and rubbery plateaus
3. **Generalized Maxwell** captures the full spectrum; single-mode Zener cannot
4. **Cross-domain validation** — same Prony terms predict both E(t) and E*(omega)
5. **Relaxation spectrum** H(tau) is a domain-independent material property

## Next Steps

- `07_dmta_tts_pipeline.ipynb`: Build master curve from raw multi-temperature data
- `08_dmta_cross_domain.ipynb`: Full cross-domain consistency analysis

In [14]:
# Cleanup
del maxwell, zener
gc.collect()
jax.clear_caches()
print("Notebook complete. Memory cleared.")

Notebook complete. Memory cleared.
